# Read Source table


In [0]:
%sql
SELECT * FROM workspace.bronze.crm_prod_info LIMIT 5

# Drop the target table if already exists

In [0]:
%sql
DROP TABLE IF EXISTS workspace.silver.crm_products

# Create the target table

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.crm_products 
(
 product_id INT,
category_id STRING,
product_number STRING,
product_name STRING,
cost INT,
product_line STRING,
start_date DATE,
end_date DATE
)USING DELTA 



In [0]:
%sql
SELECT * FROM workspace.silver.crm_products

# Do the Transformations in the original table & insert to the target table

### Product_key Parsing, product_line Trimming and Normalization, Date Casting


In [0]:
%sql
INSERT OVERWRITE workspace.silver.crm_products
SELECT
  prd_id, 
  replace(SUBSTRING(prd_key, 1, 5), '-', '_') AS category_id,
  SUBSTRING(prd_key, 7, len(prd_key)) AS product_number,
  prd_nm,
  COALESCE(prd_cost, 0) AS cost,
  CASE 
    WHEN UPPER(TRIM(prd_line)) = 'R' THEN 'ROAD'
    WHEN UPPER(TRIM(prd_line)) = 'M' THEN 'Mountain'
    WHEN UPPER(TRIM(prd_line)) = 'S' THEN 'Other_sales'
    WHEN UPPER(TRIM(prd_line)) = 'T' THEN 'Touring'
    ELSE 'n/a'
  END AS prd_line,
  CAST(prd_start_dt AS DATE) AS start_dt,
  CAST(LEAD(prd_start_dt) OVER(PARTITION BY prd_key ORDER BY prd_start_dt) -1 AS DATE) AS end_date
FROM workspace.bronze.crm_prod_info

# Sanity Checks of Target Table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_products

# View Target Table changes

In [0]:
%sql
DESCRIBE HISTORY workspace.silver.crm_products